![logo](https://github.com/donatellacea/DL_tutorials/blob/main/notebooks/figures/1128-191-max.png?raw=true)

# Model-Agnostic Interpretation with LIME

In this Notebook we will demonstrate how to use the Local Interpretable Model-Agnostic Explanations (LIME) and interpret its results.

--------

### Setup Colab environment

If you installed the packages and requirments on your own machine, you can skip this section and start from the import section.
Otherwise you can follow and execute the tutorial on your browser. In order to start working on the notebook, click on the following button, this will open this page in the Colab environment and you will be able to execute the code on your own.

<a href="https://colab.research.google.com/github/HelmholtzAI-Consultants-Munich/Zero2Hero---Introduction-to-XAI/blob/master/xai-model-agnostic/tutorial_LIME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Now that you are visualizing the notebook in Colab, run the next cell to install the packages we will use.
There are few things you should follow in order to properly set the notebook up:

1. Warning: This notebook was not authored by Google. *Click* on 'Run anyway'.
2. When the installation commands are done, there might be "Restart runtime" button at the end of the output. Please, *click* it. 

In [ ]:
!pip install lime

### Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression

from lime.lime_tabular import LimeTabularExplainer

Now, we fix the random seeds to ensure reproducible results, as we work with (pseudo) random numbers.

In [ ]:
# assert reproducible random number generation
seed = 1
np.random.seed(seed)

## The California Housing Dataset: Data Loading and Model Training

Let's use the California housing data set. The dataset is introduced in detail in the permutation importance notebook. Check it out, if you like to have more information on this dataset.

In [ ]:
calif_house_data = fetch_california_housing()
X = pd.DataFrame(calif_house_data['data'], columns = calif_house_data['feature_names'])
y = pd.DataFrame(calif_house_data['target'], columns=['Price'])

For the sake of runtime we limit ourselves to only the first 2000 samples. We will split parts of the data, so the model can not use all the available information for training. That way, we can also check performance and interpretation results on previously unseen data, mirroring the most probable practical use case.

In [ ]:
# take only first 2000 samples and convert all data frames to numpy array, which is required for LIME
feature_names = X.columns
X = X[:2000].to_numpy()
y = y[:2000].to_numpy()

# split off the test data
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=seed)
print('Number of training samples: {}'.format(X_train.shape[0]))
print('Number of test samples: {}'.format(X_test.shape[0]))


We'll create a Kernel ridge regression model with an RBF kernel to predict our *price* target variable from the other features, after they have been properly rescaled to each have zero mean and unit standard deviation.  
Don't worry for now if you are not familiar with the model. It is just meant as a protoype of a model that is not straightforward to interpret.

In [ ]:
pipe = Pipeline([
    ("preprocessing", StandardScaler()),
    ("model", KernelRidge(kernel="rbf"))
])

pipe.fit(X_train, y_train)

In [ ]:
# is the model performing reasonably on the training data?
print('Model Performance on training data: {}'.format(pipe.score(X_train, y_train)))

# is the model performing reasonably on the test data?
print('Model Performance on test data: {}'.format(pipe.score(X_test, y_test)))

Since model training and tuning is not a focus of this course, we will not try to improve the model performance further.  
**Note:** you should keep in mind that interpreting a low performing model can lead to wrong conclusions.

Now that we trained a regression model which predicts the prices relatively well, but might be a bit hard to interpet directly, we can make use of LIME to help us out.

## Now, what does my model actually think is important in the data?

### Local Interpretable Model-Agnostic Explanations (LIME)

We will try to get insights into which features are important by carrying out a method called **LIME**.

We prepared a small [video lecture](https://vimeo.com/745319036/a86f126018) for you to help you understand how LIME works.

In [ ]:
from IPython.display import VimeoVideo

VimeoVideo("745319036?h=a86f126018", width=800, height=600)

To summarize, LIME, an abbreviation for "local interpretable model agnostic explanations" is an approach that tries to deliver explanations for individual samples. It works by constructing an interpretable surrogate model (like a linear regression) to approximate predictions of a more complex model in the neighborhood of a given sample.

**Note:** this method is a **local** method which means that it does only provide explanations for individual samples, but not for a full dataset.

Now lets use Permutation Feature Importance to get some insights into the Kernel Ridge regression model we trained above.  
We first have to specify an important parameter for LIME: the *kernel_width*, which in principle determines how large the neighborhood around our sample will be. The optimal choice of this parameter is difficult and currently still an open research question and one of the main disadvantages of the method. Feel free to play around with different values and observe how the generated explanations can change.

In [ ]:
kernel_width = 0.5 # default is 0.75 * sqrt(n_features)

In [ ]:
explainer = LimeTabularExplainer(
    training_data=X_train,
    mode="regression",
    training_labels=y_train,
    feature_names=feature_names,
    feature_selection='none',  # before applying the surrogate model, one could also select features
    random_state=seed,
    sample_around_instance=True,  # NOTE: default is False!
    kernel_width=kernel_width,  
    discretize_continuous=False
)

Once we have defined the setup for LIME, we have to choose an instance for which we want to compute the local surrogate model to get explanations for the instance.

In [ ]:
# choose an instance that you want to explain
inst_idx = 0

print(f"Instance {inst_idx} of training data will be explained.")

# number of samples in the neighborhood of our point that we will use to fit
# a simpler surrogate model to explain the original complex models predictions
num_samples = 5000

instance = X_train[inst_idx]
instance_label = y_train[inst_idx]

explanation = explainer.explain_instance(
    data_row=instance,
    predict_fn=pipe.predict,
    labels=instance_label,
    model_regressor=LinearRegression(),
    num_samples=num_samples, # size of the neighborhood
)

In [ ]:
explanation.as_pyplot_figure();

All that LIME did was to fit a linear regression model to approximate the complex model's predictions. 
The dataset for creating the fit were the neighborhood samples that were randomly created around our selected instance.  
Linear regression models estimate a single parameter for each feature and those are plotted above. 
They indicate how much each feature contributes to the approximation of the complex model's predictions.
Two of the most informative features for the model are the *median income* (MedInc) and the *average number of household members* (AveOccup), which are the features with the highest absolute model coefficients. 
The positive sign of the *median income* coefficient indicates a positive relation between this feature and the target variable (*price*), i.e. a higher income leads to higher price predictions. 
On the other hand, the negative sign of the *average number of household members* coefficient suggests that as the average number of household members increases, the price tends to decrease.  

The visualization below gives us a bit of additional information than the barplots themselves:

- For the predicted value, min and max are the range of predictions of our complicated model on the neighboorhood samples
while the value itself is the prediction of the complex model at the instance we want to interpret.
- The barplot is the same as above, showing model coefficients of our simple surrogate model.
- The table on the right summarizes the feature representation of the instance we wanted to interpret.

In [ ]:
explanation.show_in_notebook()

To measure how well the simpler surrogate model is able to approximate the predictions of the more complex model, we can use any metric that summarizes the quality of the predictions that the surrogate model makes. 
This chosen metric can serve as "fidelity measure", which indicates how reliable the interpretable model is. 
Even though the choice of the fidelity measure is up to you, it is important to assess the predictive ability of the surrogate model when LIME explanations are to be used!
How much would you trust explanations delivered from a surrogate model that can not reasonably approximate the complex models predictions?

In our case, we used a linear regression model as surrogate model. A commonly used metric to quantify the goodness of fit for the linear regression model is the $R^2$ score. 
It shows how well the linear model is able to approximate the predictions of the more complex model on the neighborhood samples. $R^2$ scores closer to 1 indicate better approximations.
Our surrogate model archieves an $R^2$ score of > 0.87, indicating that the explanations given by that model can be trusted.

In [ ]:
explanation.score

<font color='green'>

#### Question 1: What is a surrogate model?

<font color='grey'>

#### Your Answer: 



<font color='green'>

#### Question 2: How is LIME using surrogate models to explain a model prediction?

<font color='grey'>

#### Your Answer: 



<font color='green'>

#### Question 3: What are the main difference to the other two methods?

<font color='grey'>

#### Your Answer: 

## Extra Material: LIME computation step by step

To get a better understanding of LIME, we will now guide you step by step through the algorthm. Even though LIME offers an easy to use API, it can be beneficial to have a quick look behind the scenes to fully understand what is going on. LIME is especially suitable here since the basic algorithm can be programmed in just a few steps.

[Cristian Arteaga](https://nbviewer.org/urls/arteagac.github.io/blog/lime.ipynb) has also prepared a nice step-by-step explanation of LIME for a 2D toy problem and we recommend to take a look at his notebook which contains nice visualizations.
Here, we are focussing on tabular data, but Christian also provides a notebook that demonstrates how LIME can work on other modalities like images as well, which is one of its big strengths!

### Step 1
First, we select an instance for which we want to explain the prediction.
We generate many normally distributed random samples that will serve as our sample neighborhood.
The samples expected value coincides with our instance to ensure sufficient similarity between the neighborhood and our instance, while the standard deviation is estimated from the training data.

Also note that we make our instance itself part of the neighborhood.

In [ ]:
np.random.seed(seed)

x = instance

# 1) generate random perturbations around our selected instance
# with given mean and standard deviation
std = X_train.std(axis=0)

# NOTE: there are two options on setting the mean of the samples.
# The default in LIME is to set it to the mean value of the training data.
# However, it may be a better idea to set the mean to the instance itself
# (in LIME this is done by sample_around_instance=True) in order
# to generate samples similar to our instance with high probability.

#mu = X_train.mean(axis=0)
mu = x

neighbors_of_x = np.random.normal(mu, std, size=(num_samples, X_train.shape[1]))

# sneek in the instance itself as part of the neighbors
neighbors_of_x[0] = x

### Step 2
We let our original model predict the outcomes of the neighborhood samples.

In [ ]:
neighbors_of_x_pipe_pred = pipe.predict(neighbors_of_x)

### Step 3
We compute the distance of each neighbor to our instance and transform it to a weight which we will later use to fit our local surrogate model.
Note that distances are computed on standardized data in order to avoid numerical instabilities.

In [ ]:
# 3) compute euclidean distance of each neighbor to x but scale the data before using the mean and standard deviation of the
#    training data
#    NOTE: distances are computed based on standardized data
#    and neighbors[0] is set to be the standardized instance itself

neighbors_of_x_scaled = pipe["preprocessing"].transform(neighbors_of_x)
x_scaled = neighbors_of_x_scaled[0]

distance_to_x = np.sum((neighbors_of_x_scaled - x_scaled)**2, axis=1)
weights = np.sqrt(np.exp(-1. * (distance_to_x / kernel_width)**2))

### Step 4
A surrogate model is fit to approximate the complex models prediction on the neighborhood samples. We will use a linear regression model since that offers fairly straightforward explanations by looking at the estimated model coefficients.
The fit will be performed again on the standardized samples! **Note:** LIME is not restricted to linear regression models and other easy-to-interpret surrogate models could be used like decision trees.

The weights computed above will serve to indicate their importance to the fit. Models with large weights are closer to our instance and should get predicted more accurately than neighbors further apart from our instance.

In [ ]:
# fit an explainable model on the scaled data to approximate the predictions of the complex model
#    on the neighborhood samples
explainable_model = LinearRegression()
explainable_model.fit(
    neighbors_of_x_scaled, 
    neighbors_of_x_pipe_pred,
    sample_weight=weights)

score = explainable_model.score(neighbors_of_x_scaled, neighbors_of_x_pipe_pred, sample_weight=weights)
print("\nModel performance", score)

### Step 5
We visualize the model coefficients to obtain a similar explanation as the LIME API offers us.

In [ ]:
# get model coefficients
coef_dict = dict(zip(feature_names, explainable_model.coef_[0]))
coef_dict_sorted = dict(sorted(coef_dict.items(), key=lambda x: np.abs(x[1]), reverse=False))
coef_df = pd.DataFrame.from_dict(coef_dict_sorted, columns=['model coefficient'], orient="index")

# plot similar as LIME provides
f, ax = plt.subplots(1, 1, figsize=(9, 7))
coef_df.plot(kind='barh', ax=ax)
plt.title('LIME - local model coefficients')
plt.xlabel('model coefficient')
plt.ylabel('feature')
plt.axvline(x=0, color='.5')
plt.subplots_adjust(left=.3)
_ = ax.set_yticklabels(coef_df.index)
ax.get_legend().remove()